In [1]:
# -*- coding=utf-8 -*-
import glob
import os
import sys
import xml.etree.ElementTree as ET
import numpy as np
from sklearn.cluster import KMeans

In [2]:
# 根文件夹
ROOT_PATH = './data/'
# 聚类的数目
CLUSTERS = 6
# 模型中图像的输入尺寸，默认是一样的
SIZE = 640

# 加载YOLO格式的标注数据
def load_dataset(path):
    jpegimages = os.path.join(path, 'train_r1')
    print(jpegimages)
    if not os.path.exists(jpegimages):
        print('no JPEGImages folders, program abort')
        sys.exit(0)
    labels_txt = os.path.join(path, 'train_r1.txt')
    if not os.path.exists(labels_txt):
        print('no labels folders, program abort')
        sys.exit(0)

#     label_file = os.listdir(labels_txt)
#     print('label count: {}'.format(len(label_file)))
    dataset = []

#     for label in label_file:
    with open(labels_txt, 'r') as f:
        txt_content = f.readlines()

    for line in txt_content:
#         line_split = line.split(' ')
#         roi_with = float(line_split[len(line_split)-2])
#         roi_height = float(line_split[len(line_split)-1])
        line = line.split()
        box = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])
        roi_with = box[:,2]-box[:,0]
        roi_height = box[:,3]-box[:,1]
        for i in range(roi_with.shape[0]):
            dataset.append(np.array([roi_with[i], roi_height[i]]))
#         dataset.append([roi_with, roi_height])
            # print([roi_with, roi_height])

    return np.array(dataset)

data = load_dataset(ROOT_PATH)
km = KMeans(n_clusters=CLUSTERS,init='k-means++')
out = km.fit_predict(data)
center = km.cluster_centers_
print(out)
# print("Accuracy: {:.2f}%".format(avg_iou(data, out) * 100))
print("Boxes:\n {}-{}".format(center[:, 0], center[:, 1]))

ratios = np.around(center[:, 0] / center[:, 1], decimals=2).tolist()
print("Ratios:\n {}".format(sorted(ratios)))

./data/train_r1
[1 2 3 ... 1 5 4]
Boxes:
 [ 90.57278481  45.99127726 122.21983914 165.01727116 182.18412698
  65.29858657]-[ 49.66218354  37.49968847  86.82037534  49.07772021 105.55873016
 100.0335689 ]
Ratios:
 [0.65, 1.23, 1.41, 1.73, 1.82, 3.36]


In [3]:
center

array([[ 90.57278481,  49.66218354],
       [ 45.99127726,  37.49968847],
       [122.21983914,  86.82037534],
       [165.01727116,  49.07772021],
       [182.18412698, 105.55873016],
       [ 65.29858657, 100.0335689 ]])

In [4]:
ratios

[1.82, 1.23, 1.41, 3.36, 1.73, 0.65]